## Exercise 1 Hello World

1. Write an MPI program displaying the number of processes used for the execution and the rank of each process.
2. Test the programs obtained with different numbers of threads for the parallel program.

**Output Example**
```shell
Hello from the rank 2 process
Hello from the rank 0 process
Hello from the rank 3 process
Hello from the rank 1 process
Parallel execution of hello_world with 4 process
```
*Note that the output order maybe different*

In [1]:
 %%file hello.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
nbOfproc = COMM.Get_size()
RANK = COMM.Get_rank()

print("Hello from the rank {} process".format(RANK))
if RANK==nbOfproc-1:
    print("Parallel execution of hello_world with {} process".format(nbOfproc))

Overwriting hello.py


In [2]:
# enter command for compile and run the program
! mpirun -n 4 python hello.py

Hello from the rank 0 process
Hello from the rank 1 process
Hello from the rank 3 process
Parallel execution of hello_world with 4 process
Hello from the rank 2 process


## Exercise 2 Sharing Data 

A common need is for one process to get data from the user, either by reading from the terminal or command line arguments, and then to distribute this information to all other processors.

Write a program that reads an integer value from the terminal and distributes the value to all of the MPI processes. Each process should print out its rank and the value it received. Values should be read until a negative integer is given as input.

You may want to use these MPI routines in your solution:
`Get_rank` `Bcast` 

**Output Example**
```shell
10
Process 0 got 10
Process 1 got 10
```

In [19]:
%%file sharing.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
nbOfproc = COMM.Get_size()
RANK = COMM.Get_rank()

if RANK==0:
    sendb=10  
else:
    sendb=None
    
recvb= COMM.bcast(sendb , root=0)
if RANK==0:
    print(recvb)
print("Process {RANK} got {data}".format(RANK=RANK, data=recvb))


Overwriting sharing.py


In [21]:
%%file sharing.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
nbOfproc = COMM.Get_size()
RANK = COMM.Get_rank()
sendb=1
recvb = 1
while recvb > 0:
    if RANK==0:
        sendb=int(input())
    recvb= COMM.bcast(sendb , root=0)
    print("Process {RANK} got {data}".format(RANK=RANK, data=recvb))
    

Overwriting sharing.py


In [20]:
! mpirun -n 2 python sharing.py

10
Process 0 got 10
Process 1 got 10


## Exercise 3 Sending in a ring (broadcast by ring)

Write a program that takes data from process zero and sends it to all of the other processes by sending it in a ring. That is, process i should receive the data and send it to process i+1, until the last process is reached.
Assume that the data consists of a single integer. Process zero reads the data from the user.
![](../data/ring.gif)

You may want to use these MPI routines in your solution:
`Send` `Recv` 

In [15]:
 %%file sending.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
nbOfproc = COMM.Get_size()
RANK = COMM.Get_rank()
tag=99
i=0
while i < nbOfproc-1:
    if RANK==i:
        sendb = 1000
        COMM.send ( sendb , dest=i+1, tag=tag )
    if RANK ==i+1:
        recvb = COMM.recv ( source=i , tag=tag )
        print ("Process {RANK} receive {recvb} from {RANKO}".format(RANK=RANK, recvb=recvb, RANKO=RANK-1))
    i=i+1 

Overwriting sending.py


In [16]:
! mpirun -n 4 python sending.py

Process 1 receive 1000 from 0
Process 2 receive 1000 from 1
Process 3 receive 1000 from 2


## Exercise 4 Matrix vector product

1. Use the `MatrixVectorMult.py` file to implement the MPI version of matrix vector multiplication.
2. Process 0 compares the result with the `dot` product.
3. Plot the scalability of your implementation. 

**Output Example**
```shell
CPU time of parallel multiplication using 2 processes is  174.923446
The error comparing to the dot product is : 1.4210854715202004e-14
```

In [54]:
 %%file MatrixVectorMult_V0.py

import numpy as np
from scipy.sparse import lil_matrix
from numpy.random import rand, seed
from numba import njit
from mpi4py import MPI


''' This program compute parallel csc matrix vector multiplication using mpi '''

COMM = MPI.COMM_WORLD
nbOfproc = COMM.Get_size()
RANK = COMM.Get_rank()

seed(42)

def matrixVectorMult(A, b, x):
    
    row, col = A.shape
    for i in range(row):
        a = A[i]
        for j in range(col):
            x[i] += a[j] * b[j]

    return 0

########################initialize matrix A and vector b ######################
#matrix sizes
SIZE = 1000
Local_size = SIZE//nbOfproc

# counts = block of each proc
#counts = 

if RANK == 0:
    A = lil_matrix((SIZE, SIZE))
    A[0, :100] = rand(100)
    A[1, 100:200] = A[0, :100]
    A.setdiag(rand(SIZE))
    A = A.toarray()
    b = rand(SIZE)
else :
    A = None
    b = None



#########Send b to all procs and scatter A (each proc has its own local matrix#####

LocalMatrix = np.zeros((Local_size, SIZE))
COMM.Scatter(A, LocalMatrix, root=0)

# Scatter the matrix A
b=COMM.bcast(b,root=0)



#####################Compute A*b locally#######################################
LocalX = np.zeros((Local_size, SIZE))


start = MPI.Wtime()
matrixVectorMult(LocalMatrix, b, LocalX)
print(LocalX)
stop = MPI.Wtime()
if RANK == 0:
    print("CPU time of parallel multiplication is ", (stop - start)*1000)

##################Gather te results ###########################################
sendcouns = len(LocalX)
sendcounts = np.array(COMM.gather(sendcouns,root=0))
if RANK == 0: 
     X = np.empty(sum(sendcounts),dtype=int)
else :
     X = None

# Gather the result into X
COMM.Gatherv(LocalX, X=(X, sendcounts,[SIZE], MPI.DOUBLE), root=0)


##################Print the results ###########################################

if RANK == 0 :
    X_ = A.dot(b)
    print("The result of A*b using dot is :", np.max(X_ - X))
    # print("The result of A*b using parallel version is :", X)
    


Overwriting MatrixVectorMult_V0.py


In [23]:
! mpirun -n 4 python MatrixVectorMult_V0.py

Traceback (most recent call last):
  File "MatrixVectorMult_V0.py", line 49, in <module>
    LocalMatrix = COMM.scatter(A, root=0)
  File "mpi4py/MPI/Comm.pyx", line 1587, in mpi4py.MPI.Comm.scatter
  File "mpi4py/MPI/msgpickle.pxi", line 823, in mpi4py.MPI.PyMPI_scatter
  File "mpi4py/MPI/msgpickle.pxi", line 161, in mpi4py.MPI.pickle_dumpv
ValueError: expecting 4 items, got 1000
^C


## Exercise 5 Calculation of π (Monte Carlo)

1. Use the `PiMonteCarlo.py` file to implement the calculation of PI using Monte Carlo.
2. Process 0 prints the result.
3. Plot the scalability of your implementation. 

In [ ]:
 %%file PiMonteCarlo_V0.py
import random 
import timeit

INTERVAL= 1000

random.seed(42)  

def compute_points():
    
    random.seed(42)  
    
    circle_points= 0

    # Total Random numbers generated= possible x 
    # values* possible y values 
    for i in range(INTERVAL**2): 
      
        # Randomly generated x and y values from a 
        # uniform distribution 
        # Rannge of x and y values is -1 to 1 
                
        rand_x= random.uniform(-1, 1) 
        rand_y= random.uniform(-1, 1) 
      
        # Distance between (x, y) from the origin 
        origin_dist= rand_x**2 + rand_y**2
      
        # Checking if (x, y) lies inside the circle 
        if origin_dist<= 1: 
            circle_points+= 1
      
        # Estimating value of pi, 
        # pi= 4*(no. of points generated inside the  
        # circle)/ (no. of points generated inside the square) 
    
     
    
    return circle_points

start = timeit.default_timer()
circle_points = compute_points()
end = timeit.default_timer()


pi = 4* circle_points/ INTERVAL**2 
print("Circle points number :",circle_points)
print("Final Estimation of Pi=", pi, "cpu time :",end-start) 

In [ ]:
! mpirun -n 4 python PiMonteCarlo_V0.py

In [46]:
%%file scatter-array.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nprocs = comm.Get_size()

if rank == 0:
    sendbuf = np.arange(15.0)

    # count: the size of each sub-task
    ave, res = divmod(sendbuf.size, nprocs)
    print(ave,res)
    count = [ave + 1 if p < res else ave for p in range(nprocs)]
    print(count)
    count = np.array(count)
    print(count)
    # displacement: the starting index of each sub-task
    displ = [sum(count[:p]) for p in range(nprocs)]
    displ = np.array(displ)
    print(displ)
else:
    sendbuf = None
    # initialize count on worker processes
    count = np.zeros(nprocs, dtype=np.int)
    displ = None

# broadcast count
comm.Bcast(count, root=0)

# initialize recvbuf on all processes
recvbuf = np.zeros(count[rank])

comm.Scatterv([sendbuf, count, displ, MPI.DOUBLE], recvbuf, root=0)

print('After Scatterv, process {} has data:'.format(rank), recvbuf)

Writing scatter-array.py


In [48]:
!mpirun -n 4 python3 scatter-array.py

In [49]:
sendbuf2 = recvbuf
recvbuf2 = np.zeros(sum(count))
comm.Gatherv(sendbuf2, [recvbuf2, count, displ, MPI.DOUBLE], root=0)

if comm.Get_rank() == 0:
    print('After Gatherv, process 0 has data:', recvbuf2)


After Gatherv, process 0 has data: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
